In [1]:
import pandas as pd
import os

os.makedirs("data_clean", exist_ok=True)

# =========================
# 1) CLEAN SPOT
# =========================
spot = pd.read_csv("data/nifty_spot_5min.csv")
spot["timestamp"] = pd.to_datetime(spot["timestamp"], errors="coerce")

spot = spot.dropna(subset=["timestamp"])
spot = spot.sort_values("timestamp").drop_duplicates(subset=["timestamp"], keep="last")

# numeric
for c in ["open","high","low","close","volume"]:
    spot[c] = pd.to_numeric(spot[c], errors="coerce")

spot = spot.dropna(subset=["open","high","low","close"])

spot.to_csv("data_clean/nifty_spot_5min_clean.csv", index=False)
print("Spot cleaned:", spot.shape)


# =========================
# 2) CLEAN FUTURES
# =========================
fut = pd.read_csv("data/nifty_futures_5min.csv")
fut["timestamp"] = pd.to_datetime(fut["timestamp"], errors="coerce")

fut = fut.dropna(subset=["timestamp"])
fut = fut.sort_values("timestamp").drop_duplicates(subset=["timestamp"], keep="last")

for c in ["open","high","low","close","volume","oi"]:
    fut[c] = pd.to_numeric(fut[c], errors="coerce")

fut = fut.dropna(subset=["open","high","low","close"])

fut.to_csv("data_clean/nifty_futures_5min_clean.csv", index=False)
print("Futures cleaned:", fut.shape)


# =========================
# 3) CLEAN OPTIONS
# =========================
opt = pd.read_csv("data/nifty_options_5min.csv")
opt["timestamp"] = pd.to_datetime(opt["timestamp"], errors="coerce")
opt["expiry"] = pd.to_datetime(opt["expiry"], errors="coerce")

opt = opt.dropna(subset=["timestamp","expiry"])
opt = opt.sort_values(["timestamp","contract_symbol"]).drop_duplicates(
    subset=["timestamp","contract_symbol"], keep="last"
)

# numeric columns
for c in ["open","high","low","close","volume","oi","strike"]:
    opt[c] = pd.to_numeric(opt[c], errors="coerce")

# keep only CE/PE
opt = opt[opt["option_type"].isin(["CE","PE"])]

opt.to_csv("data_clean/nifty_options_5min_clean.csv", index=False)
print("Options cleaned:", opt.shape)

print("\nCleaning Done! Files saved in: data_clean/")

Spot cleaned: (18462, 6)
Futures cleaned: (4125, 9)
Options cleaned: (1027188, 11)

Cleaning Done! Files saved in: data_clean/


In [2]:
import pandas as pd
import numpy as np
import os

spot_path = "data/nifty_spot_5min.csv"
fut_path  = "data/nifty_futures_5min.csv"
opt_path  = "data/nifty_options_5min.csv"

spot = pd.read_csv(spot_path)
fut  = pd.read_csv(fut_path)
opt  = pd.read_csv(opt_path)

print("Spot:", spot.shape)
print("Futures:", fut.shape)
print("Options:", opt.shape)

Spot: (18462, 6)
Futures: (4125, 9)
Options: (1027188, 11)


In [3]:
spot["timestamp"] = pd.to_datetime(spot["timestamp"], errors="coerce", utc=True).dt.tz_convert("Asia/Kolkata")
fut["timestamp"]  = pd.to_datetime(fut["timestamp"], errors="coerce", utc=True).dt.tz_convert("Asia/Kolkata")
opt["timestamp"]  = pd.to_datetime(opt["timestamp"], errors="coerce", utc=True).dt.tz_convert("Asia/Kolkata")

print("Null timestamps Spot:", spot["timestamp"].isna().sum())
print("Null timestamps Fut :", fut["timestamp"].isna().sum())
print("Null timestamps Opt :", opt["timestamp"].isna().sum())

Null timestamps Spot: 0
Null timestamps Fut : 0
Null timestamps Opt : 0


In [4]:
spot = spot.dropna(subset=["timestamp"]).sort_values("timestamp").drop_duplicates(subset=["timestamp"])
fut  = fut.dropna(subset=["timestamp"]).sort_values("timestamp").drop_duplicates(subset=["timestamp"])
opt  = opt.dropna(subset=["timestamp"]).sort_values("timestamp")

print("After cleanup:")
print("Spot:", spot.shape)
print("Fut :", fut.shape)
print("Opt :", opt.shape)

After cleanup:
Spot: (18462, 6)
Fut : (4125, 9)
Opt : (1027188, 11)


In [5]:
# Remove negative values
for c in ["open","high","low","close","volume"]:
    spot = spot[spot[c] >= 0]

# Fix OHLC logic
spot = spot[(spot["high"] >= spot["low"]) & (spot["high"] >= spot["open"]) & (spot["high"] >= spot["close"])]

# Fill missing (if any)
spot[["open","high","low","close"]] = spot[["open","high","low","close"]].ffill()
spot["volume"] = spot["volume"].fillna(0)

print("Spot nulls:", spot.isna().sum().to_dict())

Spot nulls: {'timestamp': 0, 'open': 0, 'high': 0, 'low': 0, 'close': 0, 'volume': 0}


In [6]:
for c in ["open","high","low","close","volume","oi"]:
    fut = fut[fut[c] >= 0]

fut = fut[(fut["high"] >= fut["low"]) & (fut["high"] >= fut["open"]) & (fut["high"] >= fut["close"])]

fut[["open","high","low","close"]] = fut[["open","high","low","close"]].ffill()
fut["volume"] = fut["volume"].fillna(0)
fut["oi"] = fut["oi"].fillna(method="ffill").fillna(0)

print("Futures nulls:", fut.isna().sum().to_dict())

Futures nulls: {'timestamp': 0, 'open': 0, 'high': 0, 'low': 0, 'close': 0, 'volume': 0, 'oi': 0, 'contract_symbol': 0, 'expiry': 0}


C:\Users\spart\AppData\Local\Temp\ipykernel_82880\1691905113.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fut["oi"] = fut["oi"].fillna(method="ffill").fillna(0)


In [7]:
for c in ["open","high","low","close","volume","oi","strike"]:
    if c in opt.columns:
        opt = opt[opt[c] >= 0]

opt["volume"] = opt["volume"].fillna(0)
opt["oi"] = opt["oi"].fillna(0)

print("Options nulls:", opt.isna().sum().to_dict())

Options nulls: {'timestamp': 0, 'open': 0, 'high': 0, 'low': 0, 'close': 0, 'volume': 0, 'oi': 0, 'contract_symbol': 0, 'expiry': 0, 'strike': 0, 'option_type': 0}


In [8]:
fut["contract_symbol"] = fut["contract_symbol"].astype(str)
print("Unique futures contracts:", fut["contract_symbol"].nunique())
print(fut["contract_symbol"].unique()[:10])

Unique futures contracts: 1
['NIFTY26JANFUT']


In [9]:
common_ts = set(spot["timestamp"]) & set(fut["timestamp"]) & set(opt["timestamp"])
common_ts = sorted(list(common_ts))

print("Common timestamps:", len(common_ts))

spot_aligned = spot[spot["timestamp"].isin(common_ts)].copy()
fut_aligned  = fut[fut["timestamp"].isin(common_ts)].copy()
opt_aligned  = opt[opt["timestamp"].isin(common_ts)].copy()

print("Spot aligned:", spot_aligned.shape)
print("Fut aligned :", fut_aligned.shape)
print("Opt aligned :", opt_aligned.shape)

Common timestamps: 4125
Spot aligned: (4125, 6)
Fut aligned : (4125, 9)
Opt aligned : (875483, 11)


In [10]:
# Make sure strike is numeric
opt_aligned["strike"] = pd.to_numeric(opt_aligned["strike"], errors="coerce")
opt_aligned = opt_aligned.dropna(subset=["strike"])

# Spot close map
spot_close_map = spot_aligned.set_index("timestamp")["close"]

# Add spot_close into options rows
opt_aligned["spot_close"] = opt_aligned["timestamp"].map(spot_close_map)

# ATM distance
opt_aligned["atm_dist"] = (opt_aligned["strike"] - opt_aligned["spot_close"]).abs()

# Pick ATM strike per timestamp
atm_df = opt_aligned.loc[opt_aligned.groupby("timestamp")["atm_dist"].idxmin()][["timestamp","strike"]]
atm_df = atm_df.rename(columns={"strike":"atm_strike"})

print("ATM rows:", atm_df.shape)
atm_df.head()

ATM rows: (4125, 2)


,timestamp,atm_strike
946524,2025-10-29 09:15:00+05:30,26000.0
850204,2025-10-29 09:20:00+05:30,26000.0
946526,2025-10-29 09:25:00+05:30,26000.0
326145,2025-10-29 09:30:00+05:30,26000.0
842169,2025-10-29 09:35:00+05:30,26000.0


In [11]:
opt2 = opt_aligned.merge(atm_df, on="timestamp", how="inner")

# step size approx 50 for NIFTY
step = 50

opt2["strike_level"] = (opt2["strike"] - opt2["atm_strike"]) / step

opt2 = opt2[opt2["strike_level"].isin([-2,-1,0,1,2])].copy()

print("Filtered options rows:", opt2.shape)
print("Unique strike levels:", sorted(opt2["strike_level"].unique()))

Filtered options rows: (109135, 15)
Unique strike levels: [np.float64(-2.0), np.float64(-1.0), np.float64(0.0), np.float64(1.0), np.float64(2.0)]


In [12]:
# Create CE and PE separate
ce = opt2[opt2["option_type"]=="CE"].copy()
pe = opt2[opt2["option_type"]=="PE"].copy()

# Aggregate by timestamp
ce_agg = ce.groupby("timestamp").agg(
    ce_close=("close","mean"),
    ce_oi=("oi","sum"),
    ce_vol=("volume","sum"),
).reset_index()

pe_agg = pe.groupby("timestamp").agg(
    pe_close=("close","mean"),
    pe_oi=("oi","sum"),
    pe_vol=("volume","sum"),
).reset_index()

merged = spot_aligned.merge(
    fut_aligned[["timestamp","open","high","low","close","volume","oi"]].rename(columns={
        "open":"fut_open","high":"fut_high","low":"fut_low","close":"fut_close","volume":"fut_volume","oi":"fut_oi"
    }),
    on="timestamp",
    how="left"
)

merged = merged.merge(ce_agg, on="timestamp", how="left")
merged = merged.merge(pe_agg, on="timestamp", how="left")

print("Merged shape:", merged.shape)
merged.head()

Merged shape: (4125, 18)


,timestamp,open,high,low,close,volume,fut_open,fut_high,fut_low,fut_close,fut_volume,fut_oi,ce_close,ce_oi,ce_vol,pe_close,pe_oi,pe_vol
0,2025-10-29 09:15:00+05:30,25982.00,26022.85,25966.00,25979.75,0,26400.0,26497.2,26350.0,26457.4,2795,1495,1235.0500,787350.0,3225.0,547.866667,1465750,9600
1,2025-10-29 09:20:00+05:30,25980.55,25992.50,25968.40,25988.55,0,26457.4,26474.1,26457.4,26458.5,2600,2730,1584.2375,984425.0,2925.0,552.333333,1468600,11100
2,2025-10-29 09:25:00+05:30,25990.30,26016.85,25988.15,26010.00,0,26458.5,26499.0,26458.5,26494.0,1950,4940,1610.1250,983730.0,3805.0,501.087500,1469115,11605
3,2025-10-29 09:30:00+05:30,26010.75,26025.05,25996.30,25996.35,0,26494.0,26500.0,26492.0,26492.0,1755,7345,1607.5500,986345.0,8520.0,500.275000,1468315,13365
4,2025-10-29 09:35:00+05:30,25996.90,25996.90,25963.20,25965.20,0,26492.0,26492.0,26460.0,26464.5,1625,8060,1596.8375,986540.0,505.0,512.725000,1470915,7795


In [13]:
os.makedirs("data_clean", exist_ok=True)
merged.to_csv("data_clean/nifty_merged_5min.csv", index=False)
print("Saved: data_clean/nifty_merged_5min.csv")

Saved: data_clean/nifty_merged_5min.csv


In [14]:
report_lines = []

report_lines.append("DATA CLEANING REPORT")
report_lines.append("="*50)

report_lines.append(f"Spot rows after cleaning: {spot_aligned.shape}")
report_lines.append(f"Futures rows after cleaning: {fut_aligned.shape}")
report_lines.append(f"Options rows after cleaning: {opt2.shape}")
report_lines.append(f"Merged rows: {merged.shape}")

report_lines.append("\nTimestamp alignment:")
report_lines.append(f"Common timestamps used: {len(common_ts)}")

report_lines.append("\nFutures rollover:")
report_lines.append(f"Unique futures contracts in dataset: {fut['contract_symbol'].nunique()}")
report_lines.append("NOTE: Full 1-year futures contracts not available in this Zerodha API history.")
report_lines.append("Dataset is partial, but rollover logic can work when more contracts are available.")

report_lines.append("\nATM calculation:")
report_lines.append("ATM strike calculated dynamically using nearest strike to spot close for each timestamp.")
report_lines.append("Kept only ATM ± 0/1/2 strikes.")

os.makedirs("results", exist_ok=True)
with open("results/data_cleaning_report.txt", "w") as f:
    f.write("\n".join(report_lines))

print("Saved: results/data_cleaning_report.txt")

Saved: results/data_cleaning_report.txt


In [1]:
import pandas as pd
import os

# ========= Paths =========
spot_path = "data_clean/nifty_spot_5min_clean.csv"
fut_path  = "data_clean/nifty_futures_5min_clean.csv"
opt_path  = "data_clean/nifty_options_5min_clean.csv"
merged_path = "data_clean/nifty_merged_5min.csv"

# ========= Helper =========
def check_file(path):
    print("\n==============================")
    print("Checking:", path)
    print("==============================")

    print("Exists:", os.path.exists(path))
    if not os.path.exists(path):
        return None
    
    print("Size (MB):", round(os.path.getsize(path)/1024/1024, 2))
    df = pd.read_csv(path)
    print("Rows:", len(df))
    print("Cols:", df.columns.tolist())
    print(df.head(3))
    return df

# ========= 1) Check Clean Files =========
spot = check_file(spot_path)
fut  = check_file(fut_path)
opt  = check_file(opt_path)
merged = check_file(merged_path)

# ========= 2) Timestamp Range + Nulls =========
def check_timestamp(df, name):
    if df is None: 
        return
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    print(f"\n[{name}] Timestamp Check")
    print("Min:", df["timestamp"].min())
    print("Max:", df["timestamp"].max())
    print("Null timestamps:", df["timestamp"].isna().sum())

check_timestamp(spot, "SPOT")
check_timestamp(fut, "FUTURES")
check_timestamp(opt, "OPTIONS")
check_timestamp(merged, "MERGED")

# ========= 3) Check Missing Values =========
def check_nulls(df, name):
    if df is None:
        return
    print(f"\n[{name}] Null Check (Top 10)")
    print(df.isna().sum().sort_values(ascending=False).head(10))

check_nulls(spot, "SPOT")
check_nulls(fut, "FUTURES")
check_nulls(opt, "OPTIONS")
check_nulls(merged, "MERGED")

# ========= 4) Check Futures has OI =========
if fut is not None:
    print("\n[FUTURES] OI Column Exists:", "oi" in fut.columns)
    if "oi" in fut.columns:
        print("OI Null:", fut["oi"].isna().sum())
        print("OI Min:", fut["oi"].min(), "OI Max:", fut["oi"].max())

# ========= 5) Check Options CE/PE + Strikes =========
if opt is not None:
    print("\n[OPTIONS] option_type present:", "option_type" in opt.columns)
    if "option_type" in opt.columns:
        print("Unique option types:", opt["option_type"].unique())
        print(opt["option_type"].value_counts())

    print("Unique strikes:", opt["strike"].nunique() if "strike" in opt.columns else "No strike col")

# ========= 6) Check Timestamp Alignment (Spot vs Futures vs Options) =========
if spot is not None and fut is not None and opt is not None:
    s = set(pd.to_datetime(spot["timestamp"]))
    f = set(pd.to_datetime(fut["timestamp"]))
    o = set(pd.to_datetime(opt["timestamp"]))

    print("\n[ALIGNMENT CHECK]")
    print("Common Spot ∩ Fut:", len(s & f))
    print("Common Spot ∩ Opt:", len(s & o))
    print("Common Fut  ∩ Opt:", len(f & o))
    print("Common All 3:", len(s & f & o))

# ========= 7) Check Merged has all key columns =========
if merged is not None:
    must_have = ["timestamp"]
    missing = [c for c in must_have if c not in merged.columns]
    print("\n[MERGED] Must-have columns missing:", missing)

# ========= 8) Check Merged duplicates =========
if merged is not None:
    merged["timestamp"] = pd.to_datetime(merged["timestamp"], errors="coerce")
    print("\n[MERGED] Duplicate timestamps:", merged["timestamp"].duplicated().sum())

# ========= 9) Check Sorted =========
if merged is not None:
    is_sorted = merged["timestamp"].is_monotonic_increasing
    print("\n[MERGED] Timestamp sorted:", is_sorted)

# ========= 10) Final Verdict =========
print("\nFINAL CHECK DONE. If no missing files + timestamps ok + alignment ok = READY TO SUBMIT.")


Checking: data_clean/nifty_spot_5min_clean.csv
Exists: True
Size (MB): 1.11
Rows: 18462
Cols: ['timestamp', 'open', 'high', 'low', 'close', 'volume']
                   timestamp      open      high       low     close  volume
0  2025-01-20 09:15:00+05:30  23298.90  23308.35  23219.35  23227.60       0
1  2025-01-20 09:20:00+05:30  23226.95  23231.00  23210.85  23210.85       0
2  2025-01-20 09:25:00+05:30  23211.10  23220.75  23189.70  23209.25       0

Checking: data_clean/nifty_futures_5min_clean.csv
Exists: True
Size (MB): 0.38
Rows: 4125
Cols: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'oi', 'contract_symbol', 'expiry']
                   timestamp     open     high      low    close  volume  \
0  2025-10-29 09:15:00+05:30  26400.0  26497.2  26350.0  26457.4    2795   
1  2025-10-29 09:20:00+05:30  26457.4  26474.1  26457.4  26458.5    2600   
2  2025-10-29 09:25:00+05:30  26458.5  26499.0  26458.5  26494.0    1950   

     oi contract_symbol      expiry  
0  1495   